In [10]:
import altair as alt
import numpy as np
import os
import pandas as pd
import pandas_gbq
import ssl

from google.cloud import bigquery


In [11]:
alt.data_transformers.disable_max_rows()
# pd.set_option('display.max_rows', None)

DataTransformerRegistry.enable('default')

In [3]:
# Open and read the file as a single buffer
fd = open('bq_sql/challenge_a_friend_dataframe.sql', 'r')
sql_file = fd.read()
fd.close()

In [4]:
df = pandas_gbq.read_gbq(sql_file, project_id='msds-ut-austin')

Downloading: 100%|██████████|


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476877 entries, 0 to 476876
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   user_pseudo_id          476877 non-null  object             
 1   intervention_timestamp  138727 non-null  datetime64[us, UTC]
 2   cohort                  476877 non-null  object             
 3   event_timestamp         476877 non-null  datetime64[us, UTC]
 4   event_name              476877 non-null  object             
 5   period_cohort           476877 non-null  object             
 6   exp_nos                 476877 non-null  object             
 7   variant_nos             476877 non-null  object             
 8   first_event_timestamp   476877 non-null  datetime64[us, UTC]
 9   day                     476877 non-null  Int64              
 10  week                    476877 non-null  float64            
dtypes: Int64(1), datetime64[us

In [13]:
df.event_timestamp = pd.to_datetime(df.event_timestamp, format='mixed')
df.intervention_timestamp = pd.to_datetime(df.intervention_timestamp, format='mixed')
df.first_event_timestamp = pd.to_datetime(df.first_event_timestamp, format='mixed')

# df['day'] = (df.event_timestamp - df.intervention_timestamp).dt.days
# df['week'] = np.floor((df.event_timestamp - df.intervention_timestamp).dt.days/7)

In [14]:
df.head()

,user_pseudo_id,intervention_timestamp,cohort,event_timestamp,event_name,period_cohort,exp_nos,variant_nos,first_event_timestamp,day,week
0,01BE5107A02018738DED57597F10A916,NaT,control,2018-09-29 09:31:41.320023+00:00,level_start,control-period,firebase_exp_1,0,2018-07-05 10:29:11.916011+00:00,85,12.0
1,01BE5107A02018738DED57597F10A916,NaT,control,2018-09-29 09:32:04.135006+00:00,level_start,control-period,firebase_exp_1,0,2018-07-05 10:29:11.916011+00:00,85,12.0
2,01BE5107A02018738DED57597F10A916,NaT,control,2018-09-29 09:31:43.283018+00:00,level_start,control-period,firebase_exp_1,0,2018-07-05 10:29:11.916011+00:00,85,12.0
3,01BE5107A02018738DED57597F10A916,NaT,control,2018-09-29 09:32:10.958004+00:00,level_start,control-period,firebase_exp_1,0,2018-07-05 10:29:11.916011+00:00,85,12.0
4,01BE5107A02018738DED57597F10A916,NaT,control,2018-07-05 10:33:33.661011+00:00,level_start,control-period,firebase_exp_1,0,2018-07-05 10:29:11.916011+00:00,0,0.0


In [10]:
df[df.cohort == 'control']\
    .groupby(by='user_pseudo_id')\
    .agg({'event_timestamp': ['min','max', 'count']})\
    .reset_index(col_level=1)\
    .sort_values(by=('event_timestamp','count'), ascending=False)

event_timestamp  \
                        user_pseudo_id                              min   
627   4EF964D59282E5A2C8692D30F011256B 2018-06-18 17:43:14.713011+00:00   
1117  8802C1F08F68501AB77C5097CD19CE1A 2018-06-13 02:31:40.473018+00:00   
1864  EF500E0CF578004836D1D38C8901862B 2018-06-19 06:55:14.820014+00:00   
1107  86A6C9FB1A0C566CA168149788E71EFD 2018-06-12 08:25:56.769000+00:00   
471   3B4A6FCA6DCB4C2749409DC4188A306E 2018-06-13 22:16:47.907008+00:00   
...                                ...                              ...   
532   438E2E90F1401395F41ABCC996A4FEC6 2018-06-15 17:49:14.796002+00:00   
188   1768ED05ED22B2992647C7EB51EBA3AB 2018-07-04 18:44:53.928004+00:00   
575   47F23A1C0193436E7C40B44712ABDA48 2018-09-03 07:01:47.961005+00:00   
611   4CF8FA27A7295FC44DDE986DD90DB5B8 2018-08-21 23:23:51.957010+00:00   
1487  B9CBC542BB3C218F280E39682372FFED 2018-09-26 17:52:50.509008+00:00   

                                              
                                  max  count  
627  2018-10-03 17:19:56.624292+00:00  10792  
1117 2018-10-03 20:31:03.774126+00:00   9524  
1864 2018-10-04 06:15:30.372001+00:00   7045  
1107 2018-09-28 20:40:20.675008+00:00   5766  
471  2018-10-04 01:08:04.072174+00:00   5587  
...                               ...    ...  
532  2018-06-15 17:49:14.796002+00:00      1  
188  2018-07-04 18:44:53.928004+00:00      1  
575  2018-09-03 07:01:47.961005+00:00      1  
611  2018-08-21 23:23:51.957010+00:00      1  
1487 2018-09-26 17:52:50.509008+00:00      1  

[1981 rows x 4 columns]

In [11]:
df[df.cohort == 'treatment']\
    .groupby(by='user_pseudo_id')\
    .agg({'day': ['min','max', 'count']})\
    .reset_index(col_level=1)\
    .sort_values(by=('day','min'), ascending=False)


day             
                       user_pseudo_id    min    max count
104  5EE589061E18FF6E502E31916D42B638    0.0    0.0     6
217  E63C6A09F5D06DBFA6FBF832AE8997BE    0.0    0.0     2
182  B66A869B781A5522C4911A10312EB16C    0.0    0.0     1
198  CABE7E24215A5C36862658A36AE5CE12   -1.0  102.0  4563
196  C73D0BE8A6C303863FCE487F9996028C   -1.0   22.0   194
..                                ...    ...    ...   ...
219  EAACE305D6DB2E8D80CF1E2932AE5CF4  -97.0    4.0    25
241  FE46EFC1DA3F3159AD9343EC53466F3F  -99.0    4.0   530
28   148AAE2C85735FF43C7590EC37054C17 -104.0    0.0   239
124  7566596A1D6ACA781692A7A0B89B06EF -105.0    5.0    79
21   10B0D468B5E036E16D13DB53A65F2D49 -108.0    0.0  8502

[246 rows x 4 columns]

In [12]:
df_single_user = df[df.user_pseudo_id == '959635567E419A061EECCCA478EDC718']\
    .groupby(['week', 'cleaned_event_name'])\
    .agg({
        'event_timestamp' : 'count'
    })\
    .reset_index()\
    .pivot(index='week', columns='cleaned_event_name', values='event_timestamp')\
    .reset_index()

KeyError: 'cleaned_event_name'

In [ ]:
df_single_user

cleaned_event_name,week,level_start,session_start
0,-5.0,130,10
1,-4.0,1309,28
2,-3.0,1005,26
3,-2.0,714,18
4,-1.0,700,23
5,0.0,1148,31
6,1.0,1091,35
7,2.0,1256,29
8,3.0,434,22
9,4.0,636,17


In [ ]:
# Assuming 'day' is a categorical variable, you can convert it to an ordinal scale
df_single_user['week'] = df_single_user['week'].astype('category').cat.reorder_categories(df_single_user['week'].unique(), ordered=True)

alt.Chart(df_single_user, height=700, width=700).transform_fold(
    ['level_start', 'session_start'],
    as_=['event_name', 'event_count']
).mark_line().encode(
    alt.X('week:N', title='Week'),
    alt.Y('event_count:Q', title='Value'),
    color='event_name:N',
    tooltip=['event_name:N', 'event_count:Q']
)

alt.Chart(...)